# Beautiful Soup

Es una Biblioteca de Python diseñada para recuperar información de archivos HTML y XML. Se integra con diversos analizadores sintácticos para facilitar la navegación, búsqueda y modificación del árbol de análisis, lo que suele ahorrar a los desarrolladores horas o incluso días de trabajo.

Documentación oficial: https://beautiful-soup-4.readthedocs.io/en/latest/

## Generalidades
Vamos a practicar con https://scrapepark.org/spanish/

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
# Versiones
import bs4 # Solo para el chequeo
print("Versión de BeautifulSoup:",bs4.__version__)
print("Versión de requests:", requests.__version__)

Versión de BeautifulSoup: 4.11.1
Versión de requests: 2.31.0


In [3]:
# Empezamos el scraping

# 1. Obtener el HTML
URL_BASE = 'https://scrapepark.org/courses/spanish/'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

# 2. "Parsear" ese HTML
soup = BeautifulSoup(html_obtenido, "html.parser")

In [4]:
type(soup)

bs4.BeautifulSoup

## El método `find()`

Nos permite quedarnos con la información asociada a una etiqueta de HTML

In [5]:
primer_h2 = soup.find('h2')
print(primer_h2)

<h2>¿Por qué comprar con nosotros?</h2>


In [6]:
# Solo el texto
print(primer_h2.text)

# equivalente a:
# print(soup.h2.text)

¿Por qué comprar con nosotros?


## El método `find_all()`

Busca **TODOS** los elementos de la página con esa etiqueta y devuelve una "lista" que los contiene (en realidad devuelve un objeto de la clase *bs4.element.ResultSet*).

In [7]:
h2_todos = soup.find_all('h2')
print(h2_todos)

[<h2>¿Por qué comprar con nosotros?</h2>, <h2>
                  #Novedades
                </h2>, <h2>
            Nuestros <span>productos</span>
</h2>, <h2>
            Testimonios de clientes
          </h2>, <h2 class="heading-container">
          Tabla de precios
        </h2>]


In [8]:
# ARGUMENTOS
# Si usamos el parametro limit = 1, emulamos al metodo find
h2_uno_solo = soup.find_all('h2',limit=1)
print(h2_uno_solo)

[<h2>¿Por qué comprar con nosotros?</h2>]


In [9]:
# Podemos iterar sobre el objeto
for seccion in h2_todos:
  print(seccion.text)

¿Por qué comprar con nosotros?

                  #Novedades
                

            Nuestros productos


            Testimonios de clientes
          

          Tabla de precios
        


In [10]:
# get_text() para más funcionalidades
for seccion in h2_todos:
  print(seccion.get_text(strip=True))

¿Por qué comprar con nosotros?
#Novedades
Nuestrosproductos
Testimonios de clientes
Tabla de precios


## Utilizando atributos de las etiquetas

In [11]:
# Clase
divs = soup.find_all('div', class_ = "heading-container heading-center")

for div in divs:
  print(div)
  print(" ")

<div class="heading-container heading-center" id="acerca">
<h2>¿Por qué comprar con nosotros?</h2>
</div>
 
<div class="heading-container heading-center" id="productos">
<h2>
            Nuestros <span>productos</span>
</h2>
</div>
 
<div class="heading-container heading-center">
<h3>Suscríbete para obtener descuentos y ofertas</h3>
</div>
 
<div class="heading-container heading-center">
<h2>
            Testimonios de clientes
          </h2>
</div>
 


In [12]:
# Todas las etiquetas que tengan el atributo "src"
src_todos = soup.find_all(src=True)

for elemento in src_todos:
  if elemento['src'].endswith(".jpg"):
    print(elemento)

<img alt="Parque de patinaje" src="images/slider-bg.jpg"/>
<img alt="Patineta 2" src="images/p2.jpg"/>


## Bajar todas las imagenes

In [13]:
url_imagenes = []

for i, imagen in enumerate(src_todos):

  if imagen['src'].endswith('png'):

    print(imagen['src'])
    r = requests.get(f"https://scrapepark.org/courses/spanish/{imagen['src']}")

    with open(f'imagen_{i}.png', 'wb') as f:
      f.write(r.content)

images/arrival-bg-store.png
images/p1.png
images/p3.png
images/p4.png
images/p5.png
images/p6.png
images/p7.png
images/p8.png
images/p9.png
images/p10.png
images/p11.png
images/p12.png
images/client-one.png
images/client-two.png
images/client-three.png
./images/freecodecamp-logo.png


## Tablas

In [14]:
soup.find_all('iframe')[0]['src']

'table.html'

In [15]:
# Información de tablas

URL_BASE = 'https://scrapepark.org/courses/spanish'
URL_TABLA = soup.find_all('iframe')[0]['src']

request_tabla = requests.get(f'{URL_BASE}/{URL_TABLA}')

html_tabla = request_tabla.text
soup_tabla = BeautifulSoup(html_tabla, "html.parser")
soup_tabla.find('table')

productos_faltantes = soup_tabla.find_all(['th', 'td'], attrs={'style':'color: red;'})
productos_faltantes = [talle.text for talle in productos_faltantes]

print(productos_faltantes)

['Longboard', '$80', '$85', '$90', '$62', '$150']


In [16]:
divs = soup.find_all('div', class_='detail-box')
productos = []
precios = []

for div in divs:
  if (div.h6 is not None) and ('Patineta' in div.h5.text):
    producto = div.h5.get_text(strip=True)
    precio = div.h6.get_text(strip=True).replace('$', '')
    # Se puede agregar filtros
    print(f'producto: {producto:<16} | precio: {precio}')
    productos.append(producto)
    precios.append(precio)

producto: Patineta Nueva 1 | precio: 75
producto: Patineta Usada 2 | precio: 80
producto: Patineta Nueva 3 | precio: 68
producto: Patineta Usada 4 | precio: 70
producto: Patineta Nueva 5 | precio: 75
producto: Patineta Nueva 6 | precio: 58
producto: Patineta Nueva 7 | precio: 80
producto: Patineta Nueva 8 | precio: 35
producto: Patineta Nueva 9 | precio: 165
producto: Patineta Usada 10 | precio: 54
producto: Patineta Usada 11 | precio: 99
producto: Patineta Nueva 12 | precio: 110


In [17]:
precios

['75', '80', '68', '70', '75', '58', '80', '35', '165', '54', '99', '110']

In [18]:
productos

['Patineta Nueva 1',
 'Patineta Usada 2',
 'Patineta Nueva 3',
 'Patineta Usada 4',
 'Patineta Nueva 5',
 'Patineta Nueva 6',
 'Patineta Nueva 7',
 'Patineta Nueva 8',
 'Patineta Nueva 9',
 'Patineta Usada 10',
 'Patineta Usada 11',
 'Patineta Nueva 12']

## Cambios que dependen de la URL

In [19]:
URL_BASE = "https://scrapepark.org/courses/spanish/contact"

for i in range(1,3):
  URL_FINAL = f"{URL_BASE}{i}"
  print(URL_FINAL)
  r = requests.get(URL_FINAL)
  soup = BeautifulSoup(r.text, "html.parser")
  print(soup.h5.text)

https://scrapepark.org/courses/spanish/contact1
Texto que cambia entre páginas en contacto 1 :)
https://scrapepark.org/courses/spanish/contact2
Texto que cambia entre páginas en contacto 2 :)


## Datos que no sabemos en que parte de la página se encuentran

In [20]:
# Expresiones regulares
import re

# 1. Obtener el HTML
URL_BASE = 'https://scrapepark.org/courses/spanish'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

# 2. "Parsear" ese HTML
soup = BeautifulSoup(html_obtenido, "html.parser")
# expresiones regulares
telefonos = soup.find_all(string=re.compile("\d+-\d+-\d+"))
telefonos

[' 4-444-4444']

## **Moviéndonos por el árbol**

Para saber más: https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-the-tree

In [21]:
copyrights = soup.find_all(string=re.compile("©"))
copyrights[0]

'© 2022 '

In [22]:
primer_copyright = copyrights[0]
primer_copyright.parent

<p>© 2022 <span>Todos los derechos reservados</span>.
        <a href="https://html.design/" rel="noopener noreferrer" target="_blank">Creado con Free Html Templates</a>.
      </p>

In [23]:
# # Otro ejemplo con elementos al mismo nivel
menu = soup.find(string=re.compile("MENÚ"))
# menu.parent
menu.parent.find_next_siblings()

[<ul>
 <li><a href="#">Inicio</a></li>
 <li><a href="#">Acerca</a></li>
 <li><a href="#">Servicios</a></li>
 <li><a href="#">Testimonios</a></li>
 <li><a href="#">Contacto</a></li>
 </ul>]

## Comentario sobre excepciones
https://docs.python.org/es/3/tutorial/errors.html

In [24]:
strings_a_buscar = ["MENÚ", "©", "carpincho", "Patineta"]

for string in strings_a_buscar:
  try:
    resultado = soup.find(string=re.compile(string))
    print(resultado.text)
  except AttributeError:
    print(f"El string '{string}' no fue encontrado")

MENÚ
© 2022 
El string 'carpincho' no fue encontrado

                  Patineta Nueva 1
                


## Almacenamiento de los datos

In [25]:
productos.insert(0, "productos")
precios.insert(0, "precios")
# datos = dict(zip(productos, precios))

In [26]:
datos = dict(zip(productos, precios))

In [27]:
datos.items()

dict_items([('productos', 'precios'), ('Patineta Nueva 1', '75'), ('Patineta Usada 2', '80'), ('Patineta Nueva 3', '68'), ('Patineta Usada 4', '70'), ('Patineta Nueva 5', '75'), ('Patineta Nueva 6', '58'), ('Patineta Nueva 7', '80'), ('Patineta Nueva 8', '35'), ('Patineta Nueva 9', '165'), ('Patineta Usada 10', '54'), ('Patineta Usada 11', '99'), ('Patineta Nueva 12', '110')])

In [28]:
import csv

with open('datos.csv','w') as f:
    w = csv.writer(f)
    w.writerows(datos.items())

In [29]:
import pandas as pd
dataset = pd.read_csv("C://Users//Jessica//Documents//REPOSITORIO GIT HUP//Web-Scraping-Python//datos.csv")
dataset

,productos,precios
0,Patineta Nueva 1,75
1,Patineta Usada 2,80
2,Patineta Nueva 3,68
3,Patineta Usada 4,70
4,Patineta Nueva 5,75
5,Patineta Nueva 6,58
6,Patineta Nueva 7,80
7,Patineta Nueva 8,35
8,Patineta Nueva 9,165
9,Patineta Usada 10,54


In [30]:
productos

['productos',
 'Patineta Nueva 1',
 'Patineta Usada 2',
 'Patineta Nueva 3',
 'Patineta Usada 4',
 'Patineta Nueva 5',
 'Patineta Nueva 6',
 'Patineta Nueva 7',
 'Patineta Nueva 8',
 'Patineta Nueva 9',
 'Patineta Usada 10',
 'Patineta Usada 11',
 'Patineta Nueva 12']

In [31]:
precios

['precios',
 '75',
 '80',
 '68',
 '70',
 '75',
 '58',
 '80',
 '35',
 '165',
 '54',
 '99',
 '110']

### 1. Las patinetas que salgan menos que $68

In [32]:
divs_m = soup.find_all('div', class_='detail-box')
productos_m = []
precios_m = []

for div in divs_m:
    if (div.h6 is not None) and ('Patineta' in div.h5.text):
        precio_texto = div.h6.get_text(strip=True).replace('$', '')
        precio_numero = float(precio_texto)
        if precio_numero < 68:
            producto = div.h5.get_text(strip=True)
            precio = precio_texto
            print(f'producto: {producto:<16} | precio: {precio}')
            productos_m.append(producto)
            precios_m.append(precio)

producto: Patineta Nueva 6 | precio: 58
producto: Patineta Nueva 8 | precio: 35
producto: Patineta Usada 10 | precio: 54


Conclusión: Como podemos ver los resultados, se logro mostrar las patinetas menores a $68. Lo que se volvio a recorrer div y asi ubicando los precios con una condicional de que sean menor "<" por lo que si deseamos sea mayor el simbolo se cambia ">"

### 2. Las patinetas que en su nombre tengan un numero mayor a 3

In [33]:
import re

divs_my = soup.find_all('div', class_='detail-box')
productos_my = []
precios_my = []
patron = re.compile(r'\b[4-9]\d*\b|\b\d{2,}\b')

for div in divs_my:
    if (div.h6 is not None) and ('Patineta' in div.h5.text):
        precio = div.h6.get_text(strip=True).replace('$', '')
        producto = div.h5.get_text(strip=True)
        numero_en_producto = re.findall(patron, producto)
        if any(int(numero) > 3 for numero in numero_en_producto):
            print(f'producto: {producto:<16} | precio: {precio}')
            productos_my.append(producto)
            precios_my.append(precio)

producto: Patineta Usada 4 | precio: 70
producto: Patineta Nueva 5 | precio: 75
producto: Patineta Nueva 6 | precio: 58
producto: Patineta Nueva 7 | precio: 80
producto: Patineta Nueva 8 | precio: 35
producto: Patineta Nueva 9 | precio: 165
producto: Patineta Usada 10 | precio: 54
producto: Patineta Usada 11 | precio: 99
producto: Patineta Nueva 12 | precio: 110


Conclusión: Se logro mostrar los nombres que tengan el número mayor a 3 con su respectivo precio. Se uso "import re" que es una declaración en Python que importa el módulo re, el cual proporciona soporte para expresiones regulares (regex) y  las expresiones regulares son patrones utilizados para encontrar coincidencias dentro de cadenas de texto.

### 3. Traer cualquier texto de la pagina que tenga la palabra descuento u oferta.

In [36]:
# Buscar todos los elementos que contienen las palabras "descuento" o "oferta"
textos_con_descuento_oferta = soup.find_all(text=lambda text: 'descuento' in text.lower() or 'oferta' in text.lower())
    
# Imprimir los textos encontrados
for texto in textos_con_descuento_oferta:
    print(texto.get_text(strip=True))

Descuentos 20% Off
Aprovechá nuestras ofertas.
Descuentos 20% Off
Aprovechá nuestras ofertas.
Descuentos 20% Off
Aprovechá nuestras ofertas.
Suscríbete para obtener descuentos y ofertas


Conclusión: Se hizo una busqueda usando find_all

### 4. Generar un archivo .csv con dos columnas: Una conteniendo el nombre del cliente y otra su testimonio.

In [62]:
divs = soup.find_all('div', class_='detail-box')
clientes = []
testimonios = []
# Se añade is not none para p y h5
for div in divs:
    if div.h5 is not None and div.p is not None and 'Cliente' in div.h5.text:
        cliente = div.h5.get_text(strip=True)
        testimonio = div.p.get_text(strip=True)
        print(f'cliente: {cliente:<16} | testimonio: {testimonio}')
        clientes.append(cliente)
        testimonios.append(testimonio)

cliente: Cliente 1        | testimonio: Los productos me encantaron y los precios son muy buenos. Lo recomiendo.
cliente: Cliente 2        | testimonio: ¡La calidad y variedad de patinetas es impresionante! Definitivamente volveré a comprar.
cliente: Cliente 3        | testimonio: Estoy muy conforme. Hay muchas patinetas y los diseños son fantásticos.


In [63]:
clientes

['Cliente 1', 'Cliente 2', 'Cliente 3']

In [64]:
testimonios

['Los productos me encantaron y los precios son muy buenos. Lo recomiendo.',
 '¡La calidad y variedad de patinetas es impresionante! Definitivamente volveré a comprar.',
 'Estoy muy conforme. Hay muchas patinetas y los diseños son fantásticos.']

In [65]:
clientes.insert(0,"clientes")
testimonios.insert(0,"testimonios")

In [66]:
datosClientes = dict(zip(clientes,testimonios))

In [67]:
datosClientes.items()

dict_items([('clientes', 'testimonios'), ('Cliente 1', 'Los productos me encantaron y los precios son muy buenos. Lo recomiendo.'), ('Cliente 2', '¡La calidad y variedad de patinetas es impresionante! Definitivamente volveré a comprar.'), ('Cliente 3', 'Estoy muy conforme. Hay muchas patinetas y los diseños son fantásticos.')])

In [68]:
with open('datosClientes.csv','w') as f:
    w = csv.writer(f)
    w.writerows(datosClientes.items())

Conclusión: Se realizo el proceso de almacenamiento de los datos en archivo 'csv'.